# Lab 1: Osnovna obrada teksta i označavanje vrsta riječi (POS Tagging)

## Ciljevi
Cilj ove vježbe je upoznati vas s osnovnim principima **obrade prirodnog jezika (NLP)** i praktičnim tehnikama koje se koriste za analizu tekstova. Nakon završetka vježbe trebali biste moći:

- Razumjeti osnovne korake obrade teksta u NLP-u, uključujući čišćenje podataka, tokenizaciju, lematizaciju i uklanjanje stop-riječi.
- Koristiti biblioteke `spaCy` i `NLTK` za praktičnu obradu teksta u Pythonu, uključujući prepoznavanje gramatičkih kategorija riječi.
- Izvesti **normalizaciju teksta** kako bi podaci bili pripremljeni za analizu, što uključuje uklanjanje neželjenih znakova, pretvaranje riječi u osnovni oblik i filtriranje riječi iz bilo kojeg izvora.
- Primijeniti **POS tagging** na uzorke tekstova i analizirati dobivene rezultate kako biste razumjeli strukturu jezika u tekstu.

## Očekivani ishod
Nakon vježbe, moći ćete:

- Objasniti što znače osnovni NLP pojmovi kao što su tokenizacija, lematizacija, stemming i POS tagging, te zašto su važni u obradi jezika.
- Pripremiti tekstualne podatke za analizu, uključujući njihovo čišćenje i transformaciju u format pogodan za daljnju obradu.
- Izvesti analizu vrsta riječi u tekstu i interpretirati statistike, poput broja imenica, glagola ili pridjeva, te razumjeti kakve informacije to daje o stilu i sadržaju teksta.
- Usporediti rezultate dobivene različitim alatima (`spaCy` vs `NLTK`) i razumjeti prednosti i ograničenja svakog pristupa.
- Steći temelj za naprednije NLP zadatke, kao što su analiza sentimenta (prepoznavanje tona u jeziku), prepoznavanje entiteta ili parsiranje rečenica.




---



## Instalacija potrebnih biblioteka

Koristit ćemo `spaCy` i `NLTK`, dvije najčešće korištene NLP biblioteke u Pythonu.

Web stranice:
- [Spacy 101](https://spacy.io/usage/spacy-101)
- [NTLK](https://www.nltk.org/)

`spaCy` ima unaprijed trenirane modele za različite jezike, što znači da odmah možete obraditi tekstove bez dodatnog treniranja vlastitih modela. Njegovi modeli uključuju sve od tokenizacije, lematizacije i POS tagiranja, do naprednijih funkcionalnosti poput prepoznavanja entiteta (NER) i parsiranja rečenica. spaCy je optimiziran za brzinu i praktičnu primjenu, pa se često koristi u industriji i istraživačkim projektima gdje su važni performanse i preciznost.

S druge strane, `NLTK` je prvenstveno edukacijski alat koji služi kao uvod u obradu prirodnog jezika i osnovne algoritme. Pruža veliki broj klasičnih metoda i korpusa, uključujući tokenizaciju, stemming, lematizaciju, POS tagiranje i različite tekstualne korpuse za eksperimentiranje. NLTK je vrlo pogodan za učenje i razumijevanje principa NLP-a, ali za stvarne produkcijske zadatke često je sporiji i manje precizan u odnosu na spaCy, posebno na velikim količinama podataka.



In [ ]:
# dependencies
!pip install -q -U spacy nltk matplotlib

# small English language model for spaCy
!python -m spacy download en_core_web_sm

In [ ]:
import nltk, spacy
print("spaCy version:", spacy.__version__)
print("NLTK version:", nltk.__version__)

In [ ]:
nltk.download('all')

## Što je NLP?
NLP (Natural Language Processing) je područje umjetne inteligencije koje se bavi razumijevanjem, obradom i generiranjem ljudskog jezika pomoću računala.

## Ključni koraci obrade teksta:
1. **Čišćenje i normalizacija teksta** – uklanjanje nepotrebnih znakova, brojeva, simbola.
2. **Tokenizacija** – razbijanje teksta na manje jedinice (tokene).
3. **Uklanjanje stop-riječi** – riječi bez informacijske vrijednosti (npr. "and", "the", "is").
4. **Lematizacija** – pretvaranje riječi u osnovni oblik (npr. "running" u "run").
5. **POS tagging** – označavanje svake riječi prema gramatičkoj kategoriji (imenica, glagol, pridjev...).

## Učitavanje uzorka teksta


In [ ]:
nlp = spacy.load("en_core_web_sm")

text = """Artificial intelligence is transforming industries across the world.
It enables machines to learn from data, recognize patterns, and make decisions
with minimal human intervention. However, ethical concerns continue to grow."""
doc = nlp(text)

## Tokenizacija, lematizacija i POS tagging u spaCy biblioteci

POS (Part-of-Speech) tagging je proces dodjeljivanja gramatičke oznake svakoj riječi.  
Primjeri oznaka:
- NOUN – imenica
- VERB – glagol
- ADJ – pridjev
- ADV – prilog
- PRON – zamjenica

U spaCy-ju svaka riječ ima dvije oznake:
- `pos_`: šira gramatička kategorija, vrsta riječi
- `tag_`: detaljna oznaka prema Penn Treebank standardu

Primijetite da spaCy automatski radi POS tagging prilikom obrade teksta.


In [ ]:
for token in doc:
    print(f"{token.text:<15} | {token.lemma_:<15} | {token.pos_:<15} | {token.tag_:<10} | {token.is_stop}")

## Razlika između *stemming*-a i lematizacije

U ovom primjeru vidjet ćete razliku između **stemminga** i **lematizacije** koristeći NLTK i spaCy.  
Primijetit ćete da stemming ponekad daje neprirodne oblike riječi, dok lematizacija vraća gramatički ispravne osnovne oblike.


In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# primjer
words = ["running", "studies", "better", "cars", "went", "flying"]

# Stemming pomoću NLTK PorterStemmer
stemmer = PorterStemmer()
stemmed_words = [stemmer.stem(word) for word in words]

# Lematizacija pomoću NLTK WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words_nltk = [lemmatizer.lemmatize(word) for word in words]

# Lematizacija pomoću spaCy
nlp = spacy.load("en_core_web_sm")
doc = nlp(" ".join(words))
lemmatized_words_spacy = [token.lemma_ for token in doc]

print("Originalne riječi:     ", words)
print("Stemming (NLTK):       ", stemmed_words)
print("Lematizacija (NLTK):   ", lemmatized_words_nltk)
print("Lematizacija (spaCy):  ", lemmatized_words_spacy)


## Filtriranje tokena

Sada ćemo iz početnog teksta ukloniti:
- stop-riječi (poput "is", "the", "to")
- interpunkciju
- brojeve

Ostat će samo lematizirani, čisti tokeni koji nose semantičku vrijednost.


In [ ]:
doc = nlp(text)

clean_tokens = [token.lemma_.lower() for token in doc
                if not token.is_stop and token.is_alpha]

print(clean_tokens)

## Statistička analiza POS oznaka

Možemo analizirati koliko ima imenica, glagola, pridjeva itd.  
Takva statistika daje uvid u strukturu teksta – primjerice, tehnički tekst ima više imenica i manje pridjeva.


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

pos_counts = Counter([token.pos_ for token in doc])
print(pos_counts)

plt.barh(pos_counts.keys(), pos_counts.values())
plt.title("Distribucija vrsta riječi (POS tags)")
plt.ylabel("Vrsta riječi")
plt.xlabel("Broj pojavljivanja")
plt.show()

## Usporedba sa NLTK

Biblioteka **NLTK (Natural Language Toolkit)** bila je jedan od prvih ozbiljnih alata za rad s prirodnim jezikom u Pythonu i dugo je služila kao standard u akademskim okruženjima.  
NLTK ima vlastitu implementaciju POS taggera koja koristi **Penn Treebank oznake** – to je skup standardiziranih kratkih oznaka (npr. `NN` za imenice, `VB` za glagole, `JJ` za pridjeve). Ove oznake potječu iz jednog od najpoznatijih lingvističkih korpusa na engleskom jeziku.

U ovoj vježbi koristimo NLTK-ov ugrađeni tagger `averaged_perceptron_tagger`, koji se temelji na statističkoj metodi perceptrona i radi vrlo brzo, ali nije uvijek dosljedan u svim kontekstima. Za kraće tekstove je sasvim dovoljan.

Kad usporedite rezultate **spaCyja** i **NLTK-a**, primijetit ćete da se oznake ponekad ne poklapaju. To je posljedica nekoliko razlika u načinu rada ovih alata:

- **Tokenizacija**: spaCy koristi vlastiti tokenizator koji prepoznaje skraćenice, brojeve i interpunkciju inteligentnije od NLTK-ovog osnovnog tokenizatora.  
- **Modeli**: spaCy koristi unaprijed trenirani model temeljen na modernijim metodama (npr. kontekstualni vektori i neuronske mreže), dok NLTK koristi starije, *rule-based* i statističke pristupe.  
- **Oznake i korpusi**: iako se oba oslanjaju na Penn Treebank sustav, implementacije se razlikuju jer su modeli trenirani na različitim verzijama i podskupovima korpusa.  



In [ ]:
from nltk import pos_tag, word_tokenize

tokens = word_tokenize(text)
nltk_tags = pos_tag(tokens)

# spaCy
spacy_tags = [(token.text, token.pos_) for token in doc]

print("Usporedba NLTK i spaCy tagova:")
print("-" * 50)
print(f"{'Word':<15} | {'NLTK Tag':<10} | {'spaCy Tag':<10}")
print("-" * 50)

# Primijetite da se tagovi ponekad razlikuju ovisno o tome koju biblioteku koristimo
spacy_tag_dict = {token.text: token.tag_ for token in doc}

for word, nltk_tag in nltk_tags:
    spacy_tag = spacy_tag_dict.get(word, "N/A")
    print(f"{word:<15} | {nltk_tag:<10} | {spacy_tag:<10}")

### Primijetite da je NLTK prepoznao riječ 'recognize' kao imenicu, dok je spaCy prepoznao da je to glagol.

## Pomoćne funkcije za zadatke

In [ ]:
import requests
import re

def fetch_html(url, filename):
    response = requests.get(url)
    response.encoding = 'utf-8'
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(response.text)
    return response.text

def load_html(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()

def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def clean_and_split(text):
    text = re.sub(r'>(\s*)<', '> <', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


## Zadatak 1

Preuzmi HTML sadržaj s poveznice [https://www.coursera.org/articles/natural-language-processing](https://www.coursera.org/articles/natural-language-processing) koristeći Python biblioteku `requests`. Zatim, pomoću regularnih izraza (*regex*-a), pronađi i prebroji koliko puta se pojavljuju riječi **NLP** i fraza **natural language** (ignoriraj velika/mala slova). Prebroji koliko puta se u tekstu pojavljuju HTML tagovi `<h1>`, `<h2>`, `<h3>` i `<p>`. Na kraju, pronađi i prebroji koliko eksternih referenci postoji tako da izvučeš sve linkove (https://...). Sve rezultate ispiši na ekranu. Pomoćne funkcije nalaze se u ćeliji iznad.

In [ ]:
url = "https://www.coursera.org/articles/natural-language-processing"
filename = "article.html"

# Dohvaćanje HTMLa sa stranice i spremi na disk
html = fetch_html(url, filename)

# Parsiranje HTMLa regularnim izrazima za tražene pojmove/tipove tagova
nlp_count = len(re.findall(r'\bNLP\b', html, re.IGNORECASE))
natural_language_count = len(re.findall(r'natural language', html, re.IGNORECASE))

h1_count = len(re.findall(r'<h1', html, re.IGNORECASE))
h2_count = len(re.findall(r'<h2', html, re.IGNORECASE))
h3_count = len(re.findall(r'<h3', html, re.IGNORECASE))
p_count  = len(re.findall(r'<p',  html, re.IGNORECASE))

external_links_count = len(re.findall(r'href=[\'"]https', html))

# Ispis rezultata na ekranu
print(f"Broj pojavljivanja 'NLP': {nlp_count}")
print(f"Broj pojavljivanja 'natural language': {natural_language_count}")
print(f"Broj <h1> tagova: {h1_count}")
print(f"Broj <h2> tagova: {h2_count}")
print(f"Broj <h3> tagova: {h3_count}")
print(f"Broj <p> tagova: {p_count}")
print(f"Broj eksternih linkova: {external_links_count}")

## Zadatak 2

Učitaj HTML sadržaj članka koji si prethodno preuzeo (ili koristi istu tekstualnu varijablu). Uz pomoć custom funkcija za parsiranje i čišćenje teksta, izdvoji glavni naslov članka (element `<h1>`), sve podnaslove (`<h2>` i `<h3>`) te tekst iz svih paragrafa (`<p>`). Paragrafe spoji u jedan cjeloviti string tako da koristiš novi red kao separator i ispiši prvih 500 znakova rezultirajućeg teksta. Pronađi i ispiši prvih pet stavki nabrajanja (element `<li>`), ako postoje. Sve tražene podatke ispiši na ekranu.


In [ ]:
filename = "article.html"
html = load_html(filename)

# Glavni naslov <h1>
main_title = re.search(r'<h1.*?>(.*?)</h1>', html, re.DOTALL | re.IGNORECASE)
print("Glavni naslov:", strip_tags(main_title.group(1)) if main_title else "Nema")

# Podnaslovi <h2> i <h3>
h2s = [strip_tags(x) for x in re.findall(r'<h2.*?>(.*?)</h2>', html, re.DOTALL | re.IGNORECASE) if strip_tags(x)]
h3s = [strip_tags(x) for x in re.findall(r'<h3.*?>(.*?)</h3>', html, re.DOTALL | re.IGNORECASE) if strip_tags(x)]
print("Podnaslovi (h2):", h2s)
print("Podnaslovi (h3):", h3s)

# Paragrafski tekst <p>
paragraphs = re.findall(r'<p.*?>(.*?)</p>', html, re.DOTALL | re.IGNORECASE)
full_text = '\n'.join([clean_and_split(p) for p in paragraphs if clean_and_split(p)])
print("Prvih 500 znakova teksta:", full_text[:500])

# Nabrajanja (li): samo prave <li> tagove
li_items = re.findall(r'<li(?:\s|>)[^>]*>(.*?)</li>', html, re.DOTALL | re.IGNORECASE)
if li_items:
    print("Nabrojane stavke (li):")
    for item in li_items[:5]:
        print("-", strip_tags(item))
else:
    print("Nema nabrajanja u članku.")


## Zadatak 3

Analiziraj tekst članka s pomoću `spaCy` biblioteke. Filtriraj tokene (bez stopwords, interpunkcije i praznih), prikaži tablicu za prvih 30 tokena (riječ, lema, POS), ispiši broj svih riječi, jedinstvenih riječi i rečenica. Prikaži tri najčešća POS taga i nacrtaj bar chart njihovih frekvencija. Koristi definirane funkcije za dohvat i čišćenje teksta.


In [ ]:
import spacy
from collections import Counter
import matplotlib.pyplot as plt

filename = "article.html"
html = load_html(filename)

paragraphs = re.findall(r'<p.*?>(.*?)</p>', html, re.DOTALL | re.IGNORECASE)
full_text = '\n'.join([clean_and_split(p) for p in paragraphs if clean_and_split(p)])

# NLP analiza sa spaCy
nlp = spacy.load('en_core_web_sm')
doc = nlp(full_text)

# Filtrirani tokeni (bez stopwords, interpunkcije, praznih)
tokens = [token for token in doc if not token.is_stop and not token.is_punct and not token.is_space]

# Tablica za prvih 30 tokena
print(f"{'Riječ':15} {'Lema':15} {'POS':8}")
for token in tokens[:30]:
    print(f"{token.text:15} {token.lemma_:15} {token.pos_:8}")

num_words = len([token for token in doc if not token.is_punct and not token.is_space])
num_unique_words = len(set(token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space))
num_sentences = len(list(doc.sents))
print("\nBroj riječi:", num_words)
print("Broj jedinstvenih riječi:", num_unique_words)
print("Broj rečenica:", num_sentences)

# Tri najčešća POS taga
pos_counts = Counter(token.pos_ for token in tokens)
print("\nTri najčešća POS taga:")
for pos, count in pos_counts.most_common(3):
    print(f"{pos}: {count}")

# Bar chart POS tagova
labels, values = zip(*pos_counts.items())
plt.figure(figsize=(8,4))
plt.bar(labels, values)
plt.title("Frekvencija POS tagova")
plt.ylabel("Broj pojavljivanja")
plt.xlabel("POS tag")
plt.show()


## Opcionalno: POS tagging na hrvatskom jeziku
spaCy podržava hrvatski model `hr_core_news_sm`.

Usporedite dosadašnje rezultate s engleskim tekstom.


In [ ]:
!python -m spacy download hr_core_news_sm
import spacy
nlp_hr = spacy.load("hr_core_news_sm")

text_hr = "Umjetna inteligencija mijenja način na koji živimo i radimo."
doc_hr = nlp_hr(text_hr)

for token in doc_hr:
    print(f"{token.text:<15} | lemma: {token.lemma_:<15} | POS: {token.pos_:<10}")


## Zaključak

U ovoj vježbi ste naučili kako se tekst može pripremiti za analizu, odnosno kako ga **normalizirati**, očistiti od nepotrebnih znakova i stop-riječi, te pretvoriti u oblik pogodan za daljnju obradu. Također ste primijenili **POS tagging** pomoću biblioteka spaCy i NLTK i vidjeli kako se svaka riječ može označiti odgovarajućom gramatičkom kategorijom. Na kraju ste naučili **interpretirati rezultate** i dobili uvid u strukturu jezika u tekstu, primjerice, koje vrste riječi prevladavaju i što to govori o stilu i sadržaju teksta.

### Što dalje?
- Analizirajte tekstove različitih domena (npr. vijesti vs. književnost, tehnički tekstovi vs. blogovi) i usporedite njihove POS distribucije kako biste uočili stilističke i sadržajne razlike.
- Napravite **grafičku usporedbu više dokumenata** s obzirom na njihovu POS strukturu, što može pomoći u vizualizaciji dominantnih vrsta riječi i razlikovanju tipova tekstova.
- Istražite razlike između modela za različite jezike ili različite alate za POS tagging kako biste razumjeli kako izbor tehnologije utječe na analizu.
